In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from prettytable import PrettyTable
from sklearn.svm import SVC
import tensorflow as tf
import datetime
from random import randint
from gensim.models import word2vec

In [0]:
word2vec_model_name = "/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/word_embedding/fasttext/commen_docid_removed300_5"

num_features = 300
max_sentence_length = 50

batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 1000

labels = tf.placeholder(tf.int32, [batchSize, numClasses])
data = tf.placeholder(tf.float32, [batchSize, max_sentence_length, num_features])

In [0]:
def main():
    start_time = time.time()
    run_cross_val()
    end_time = time.time()
    print("Time taken for the process: " + str(end_time - start_time))
    return


def convert_to_vectors():
    comments = pd.read_csv("/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/corpus/analyzed/comments_tagged_remove_all_punc_keep_question.csv", ";")
    data_vectors, data_labels = comments_to_vectors(comments)

    np.save('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/vectors_from_W2vRnnCrossVal/data_vectors_common_docid_removed.npy', data_vectors)
    np.save('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/vectors_from_W2vRnnCrossVal/data_labels_common_docid_removed.npy', data_labels)


def load_vectors():
    data_vectors = np.load('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/vectors_from_W2vRnnCrossVal/data_vectors_common_docid_removed.npy')
    data_labels = np.load('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/vectors_from_W2vRnnCrossVal/data_labels_common_docid_removed.npy')
    return data_vectors, data_labels


def comments_to_vectors(data):
    model = word2vec.Word2Vec.load(word2vec_model_name)
    comment_vectors = []
    comment_labels = []
    for comment in data["comment"]:
        comment_vectors.append(get_sentence_vector(model, comment))
    for label in data["label"]:
        if label == "POSITIVE":
            comment_labels.append([0, 1])
        else:
            comment_labels.append([1, 0])
    return np.array(comment_vectors), comment_labels


def get_sentence_vector(model, sentence):
    sentence_vector = np.zeros([max_sentence_length, num_features])
    counter = 0
    index2word_set = set(model.wv.index2word)
    for word in sentence.split():
        if word in index2word_set:
            sentence_vector[counter] = model[word]
            counter += 1
            if (counter == max_sentence_length):
                break
        else:
            print("word not in word2vec model: " + word)
    return sentence_vector


def get_batch(size, data, label):
    batch_data = np.empty((size, max_sentence_length, num_features), dtype=float)
    batch_label = []
    for i in range(size):
        random_int = randint(0, len(data) - 1)
        batch_data[i] = data[random_int]
        batch_label.append(label[random_int])
    return batch_data, batch_label


def get_batch_order(size, data, label, batch_no):
    batch_data = data[batch_no * size : (batch_no + 1) * size]
    batch_label = label[batch_no * size : (batch_no + 1) * size]
    return batch_data, batch_label


def neural_network_model():
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)

    lstm_cell = tf.contrib.rnn.DropoutWrapper(cell=lstm_cell, output_keep_prob=0.75)
    value, _ = tf.nn.dynamic_rnn(lstm_cell, data, dtype=tf.float32)

    weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
    bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
    value = tf.transpose(value, [1, 0, 2])
    last = tf.gather(value, int(value.get_shape()[0]) - 1)
    prediction = (tf.matmul(last, weight) + bias)

    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    prediction_values = tf.argmax(prediction, 1)

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    return loss, accuracy, prediction_values, optimizer


def train_neural_network(loss, accuracy, optimizer, train_data, train_labels):
    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    tf.summary.scalar('Loss', loss)
    tf.summary.scalar('Accuracy', accuracy)
    merged = tf.summary.merge_all()
    logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
    writer = tf.summary.FileWriter(logdir, sess.graph)

    for i in range(iterations):
        #Next Batch of reviews
        next_batch, next_batch_labels = get_batch(batchSize, train_data, train_labels)
        sess.run(optimizer, {data: next_batch, labels: next_batch_labels})

        #Write summary to Tensorboard
        if (i % 499 == 0):
            summary = sess.run(merged, {data: next_batch, labels: next_batch_labels})
            writer.add_summary(summary, i)

        #Save the network every 10,000 training iterations
        if (i % 499 == 0 and i != 0):
            save_path = saver.save(sess, "/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    writer.close()


def measure_neural_network(accuracy, prediction_values, test_data, test_labels):
    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    saver.restore(sess, tf.train.latest_checkpoint('/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/'))

    overall_accuracy = 0
    all_predictions = []
    test_iterations = 20
    for i in range(test_iterations):
        next_batch, next_batch_labels = get_batch_order(batchSize, test_data, test_labels, i)
        accuracy_this_batch = (sess.run(accuracy, {data: next_batch, labels: next_batch_labels})) * 100
        predictions_this_batch = sess.run(prediction_values, {data: next_batch, labels: next_batch_labels})
        overall_accuracy = overall_accuracy + accuracy_this_batch
        all_predictions = all_predictions + predictions_this_batch.tolist()
        print("Accuracy for this batch:", accuracy_this_batch)

    true_labels = tf.argmax(test_labels, 1).eval()
    precision = precision_score(true_labels.tolist()[0:batchSize * test_iterations], all_predictions)
    f1 = f1_score(true_labels.tolist()[0:batchSize * test_iterations], all_predictions)
    recall = recall_score(true_labels.tolist()[0:batchSize * test_iterations], all_predictions)
    overall_accuracy = overall_accuracy / (test_iterations * 100)
    print(confusion_matrix(true_labels.tolist()[0:batchSize * test_iterations], all_predictions).ravel())

    all_test = true_labels.tolist()[0:batchSize * test_iterations]
    return overall_accuracy, precision, recall, f1, all_predictions, all_test


def run_cross_val():
    all_predictions = []
    all_used_test_labels = []
    # w2v_model_path = "../../../corpus/analyzed/saved_models/"
    comments = pd.read_csv("/content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/corpus/analyzed/comments_tagged_remove_all_punc_keep_question.csv", ";")
    pretty_table = PrettyTable(["Algorithm", "Accuracy", "Precision", "Recall", "F1_Score"])

    # convert_to_vectors()
    data_vectors, data_labels = load_vectors()

    print("Running tesnsorflow simulation.....")


    i = 1
    kf = KFold(n_splits=10)
    kf.get_n_splits(comments)
    for train_index, test_index in kf.split(data_vectors):
        train_data_comments, test_data_comments = data_vectors[train_index], data_vectors[test_index]
        train_data_labels, test_data_labels = data_labels[train_index], data_labels[test_index]


        tf.reset_default_graph()

        global labels
        global data
        labels = tf.placeholder(tf.int32, [batchSize, numClasses])
        data = tf.placeholder(tf.float32, [batchSize, max_sentence_length, num_features])

        loss, accuracy, prediction_values, optimizer = neural_network_model()
        train_neural_network(loss, accuracy, optimizer, train_data_comments, train_data_labels)
        accuracy, precision, recall, f1, predictions, used_test_labels = measure_neural_network(accuracy, prediction_values, test_data_comments, test_data_labels)

        all_predictions = all_predictions + predictions
        all_used_test_labels = all_used_test_labels + used_test_labels

        i = i + 1
        evaluation_metrics(used_test_labels, predictions, pretty_table, "iteration" + str(i))

    evaluation_metrics(all_used_test_labels, all_predictions, pretty_table, "final")
    print(pretty_table)
    print_confusion_matrix(all_used_test_labels, all_predictions)



def fit_models(vectorizer, train_data_comments, test_data_comments, train_data_labels, test_data_labels):
    pretty_table = PrettyTable(["Algorithm", "Accuracy", "Precision", "Recall", "F1_Score"])

    vectorized_train_comments = vectorizer.fit_transform(train_data_comments)
    vectorized_test_comments = vectorizer.transform(test_data_comments)

    model = SVC(C=1, kernel='linear')
    model = model.fit(vectorized_train_comments, train_data_labels)
    predictions = model.predict(vectorized_test_comments)
    evaluation_metrics(test_data_labels, predictions, pretty_table, "SVM")
    print_confusion_matrix(test_data_labels, predictions)

    print(pretty_table)
    print("")
    return predictions


def evaluation_metrics(true_sentiment, predicted_sentiment, pretty_table, algorithm):
    label_binarizer = preprocessing.LabelBinarizer()
    label_binarizer.fit(['NEGATIVE', 'POSITIVE'])
    test_labels = label_binarizer.transform(true_sentiment)
    predict_labels = label_binarizer.transform(predicted_sentiment)
    accuracy_str = str(accuracy_score(true_sentiment, predicted_sentiment))
    precision_str = str(precision_score(true_sentiment, predicted_sentiment))
    recall_str = str(recall_score(true_sentiment, predicted_sentiment))
    f1_score_str = str(f1_score(true_sentiment, predicted_sentiment))
    pretty_table.add_row([algorithm, accuracy_str, precision_str, recall_str, f1_score_str])
    return


def print_confusion_matrix(label, prediction):
    cf_matrix = confusion_matrix(label, prediction)
    print(cf_matrix.ravel())
    print(cf_matrix)

In [9]:
convert_to_vectors()

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


word not in word2vec model: කොළෙන්
word not in word2vec model: පින්වැඩි
word not in word2vec model: වරදටනම්
word not in word2vec model: ිනයම
word not in word2vec model: කෙනෙන්ක්ටම
word not in word2vec model: එකකකට
word not in word2vec model: මොරදෙන්නන්ට
word not in word2vec model: හාඟන්නු
word not in word2vec model: හිඟන්නගෙන්
word not in word2vec model: හිඟාකෑමත්
word not in word2vec model: හිගන්නොනේ
word not in word2vec model: අනුග්‍රහයවත්
word not in word2vec model: එකමුතුකමෙන්වත්
word not in word2vec model: නොසැළිකිලිමත්කමේ
word not in word2vec model: රියැදුරාගේ
word not in word2vec model: දෙමවුපියොද?
word not in word2vec model: නොදැනුනානම්
word not in word2vec model: නරුනත්
word not in word2vec model: සරදියෙල්වත්
word not in word2vec model: සමරනවානම්
word not in word2vec model: කෑවගේ
word not in word2vec model: ඉ්සසරහට
word not in word2vec model: සරදියෙල්ල
word not in word2vec model: කුනුවෙන්න
word not in word2vec model: අරිනවද?
word not in word2vec model: අසීරුතාවයකින්ද
word not 

In [11]:
main()

Running tesnsorflow simulation.....


/tensorflow-1.15.0/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-499
saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
Accuracy for this batch: 79.16666865348816
Accuracy for this batch: 87.5
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 87.5
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 100.0
Accuracy for this batch: 54.16666865348816
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 75.0
Accuracy for this batch: 79.16666865348816
Accura

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/tensorflow-1.15.0/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-499
saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
Accuracy for this batch: 66.66666865348816
Accuracy for this batch: 75.0
Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 87.5
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 75.0
Accuracy for this batch: 75.0
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 100.0
Accuracy for this b

...

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/tensorflow-1.15.0/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-499
saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 87.5
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 75.0
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 91.66666865348816
Accuracy for this batch:

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/tensorflow-1.15.0/python3.6/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-499
saved to /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/University/FYP/Sentiment Analysis/Implementation/Sentiment-tagger/RNN/models/from_fasttext/5/pretrained_lstm.ckpt-998
Accuracy for this batch: 87.5
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 87.5
Accuracy for this batch: 91.66666865348816
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 100.0
Accuracy for this batch: 75.0
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 79.16666865348816
Accura

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Results from 100 epochs

+-------------+--------------------+--------------------+--------------------+--------------------+
|  Algorithm  |      Accuracy      |     Precision      |       Recall       |      F1_Score      |
+-------------+--------------------+--------------------+--------------------+--------------------+
|  iteration2 | 0.7479166666666667 | 0.8456375838926175 |       0.5625       | 0.6756032171581771 |
|  iteration3 |      0.74375       | 0.8705882352941177 | 0.5943775100401606 | 0.7064439140811456 |
|  iteration4 | 0.8083333333333333 | 0.9336283185840708 | 0.7326388888888888 | 0.8210116731517509 |
|  iteration5 | 0.7666666666666667 | 0.9148936170212766 | 0.6417910447761194 | 0.7543859649122806 |
|  iteration6 |       0.775        | 0.8571428571428571 | 0.631578947368421  | 0.7272727272727273 |
|  iteration7 | 0.7708333333333334 | 0.8963414634146342 |       0.6125       | 0.7277227722772278 |
|  iteration8 | 0.8020833333333334 | 0.8812785388127854 | 0.7366412213740458 | 0.8024948024948024 |
|  iteration9 | 0.7854166666666667 | 0.9380952380952381 | 0.686411149825784  | 0.7927565392354124 |
| iteration10 | 0.8041666666666667 | 0.8243243243243243 | 0.6421052631578947 | 0.7218934911242604 |
| iteration11 | 0.8083333333333333 | 0.7380952380952381 | 0.6118421052631579 | 0.6690647482014388 |
|    final    |      0.78125       | 0.878393665158371  | 0.6503350083752094 | 0.7473532242540905 |
+-------------+--------------------+--------------------+--------------------+--------------------+
[2197  215  835 1553]
[[2197  215]
 [ 835 1553]]